https://fluxml.ai/Flux.jl/stable/models/recurrence/

https://github.com/FluxML/model-zoo/

https://denizyuret.github.io/Knet.jl/latest/rnn/

https://colah.github.io/posts/2015-08-Understanding-LSTMs/ find rus

https://habr.com/ru/company/wunderfund/blog/331310/

In [9]:
using Flux
using Flux: onehot, chunk, batchseq, throttle, crossentropy
using StatsBase: wsample
using Base.Iterators: partition

In [39]:
isfile("input.txt") ||
  download("https://cs.stanford.edu/people/karpathy/char-rnn/shakespeare_input.txt",
           "input.txt")

"input.txt"

Больше текстовых данных

In [1]:
# http://tululu.org/a16884/6/
pth = "C:/Users/User/Downloads/tutorial/files/" # files # shakespeare # Tolstoy
fnames = readdir(pth)

20-element Array{String,1}:
 "Adolbf_Gitler_(Tom_1).txt"                                                                              
 "Aforizmy.txt"                                                                                           
 "Akademija_pana_Kljaksy_(per._M.Landman).txt"                                                            
 "anna-karenina.txt"                                                                                      
 "A_mozhet,_v_jetot_raz.txt"                                                                              
 "Dbjavol.txt"                                                                                            
 "Detstvo.txt"                                                                                            
 "Hadzhi-Murat.txt"                                                                                       
 "Jekstremalbnoe_programmirovanie.txt"                                                                    
 "Sevasto

Все данные в одну строку

In [2]:
str = prod( [read(pth*fname, String) for fname in fnames ] );
length(str)

16899266

In [3]:
prod(sort([unique(str)..., '_']) )

"\t\n\r !\"#\$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^__`abcdefghijklmnopqrstuvwxyz{|}~ §©«\uad®°±·»ÇÊàâäåçèéêëíîïôöùûüœπЁАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюяёїњ–—‘’“”„•…№™\ufeff"

избавляемся от лишних символов

In [5]:
trash = "\t\r\"#\$%&'()*·+-/0123456789:;<=>@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^__`abcdefghijklmnopqrstuvwxyz{|}~§©«\uad®°±»ÇÊàâäåçèéêëíîïôöùûüœπЁЪЫЬёїњ–—‘’“”„•…№™\ufeff"

for c in collect(trash)
    str = replace(str, c => "");
end

alstr = prod(sort([unique(str)..., '_']) )

"\n !,.?_ АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЮЯабвгдежзийклмнопрстуфхцчшщъыьэюя"

In [9]:
alstr[2]

' ': ASCII/Unicode U+0020 (category Zs: Separator, space)

In [7]:
texta = collect(str)
#str = nothing
length(texta)

16375384

In [10]:
#texta = collect(String(read("C:\\Users\\User\\Downloads\\intro_to_ML\\input.txt")))
alphabet = [unique(texta)..., '_']
texta = map(ch -> onehot(ch, alphabet), texta)
stopa = onehot('_', alphabet)
length(alphabet)

69

In [34]:
N = length(alphabet)
seqlen = 50
nbatch = 50

Xs = collect(partition(batchseq(chunk(texta, nbatch), stopa), seqlen))
Ys = collect(partition(batchseq(chunk(texta[2:end], nbatch), stopa), seqlen));

## LSTM

In [35]:
m = Chain(
  LSTM(N, 128),
  LSTM(128, 128),
  Dense(128, N),
  softmax)

Chain(Recur(LSTMCell(60, 128)), Recur(LSTMCell(128, 128)), Dense(128, 60), softmax)

In [ ]:
# загрузить параметры обученой модели с диска
using BSON: @load

@load "mymodel.bson" weights

Flux.loadparams!(m, weights)

In [9]:
m = Chain(
  LSTM(N, 256),
  LSTM(256, 128),
  LSTM(128, 128), # 113min
  Dense(128, N),
  softmax)

Chain(Recur(LSTMCell(69, 256)), Recur(LSTMCell(256, 128)), Recur(LSTMCell(128, 128)), Dense(128, 69), softmax)

In [51]:
?truncate

search: truncate



```
truncate(file, n)
```

Resize the file or buffer given by the first argument to exactly `n` bytes, filling previously unallocated space with '\0' if the file or buffer is grown.

# Examples

```jldoctest
julia> io = IOBuffer();

julia> write(io, "JuliaLang is a GitHub organization.")
35

julia> truncate(io, 15)
IOBuffer(data=UInt8[...], readable=true, writable=true, seekable=true, append=false, size=15, maxsize=Inf, ptr=16, mark=-1)

julia> String(take!(io))
"JuliaLang is a "

julia> io = IOBuffer();

julia> write(io, "JuliaLang is a GitHub organization.");

julia> truncate(io, 40);

julia> String(take!(io))
"JuliaLang is a GitHub organization.\0\0\0\0\0"
```


In [36]:
function loss(xs, ys)
  l = sum(crossentropy.(m.(gpu.(xs)), gpu.(ys)))
  Flux.truncate!(m)
  return l
end

loss (generic function with 1 method)

In [37]:
opt = ADAM(0.01)
tx, ty = (Xs[5], Ys[5])
evalcb = () -> @show loss(tx, ty)

#17 (generic function with 1 method)

In [ ]:
@time Flux.train!(loss, params(m), zip(Xs, Ys), opt,
            cb = throttle(evalcb, 30))

In [19]:
9561.935529 /60 # минут

159.36559215

In [12]:
function sample(m, alphabet, len)
  #m = cpu(m)
  Flux.reset!(m)
  buf = IOBuffer()
  c = rand(alphabet)
  for i = 1:len
    write(buf, c)
    c = wsample(alphabet, m( onehot(c, alphabet) ).data )
  end
  return String(take!(buf))
end

sample (generic function with 1 method)

In [49]:
sample(m, alphabet, 1000) |> println

QUEN:
Chiet? The buswievest by his seld me not report.
Good eurronish too in me will lide upon the name;
Nor pain eat, comes, like my nature is night.

GRUMIO:
What for the Patrople:
While Antony ere the madable sut killing! I think, bull call.
I have what is that from the mock of France:
Then, let me?

CAMILLE:
Who! we break be what you known, shade well?

PRINCE HOTHEM:
If I kiss my go reas, if he will leave; which my king myself.

BENEDICH:
The aunest hathing rouman can as? Come, my arms and haste.
This weal the humens?
Come sifen, shall as some best smine? You would hain to all make on,
That that herself: whom will you come, lords and lafe to overwark the could king to me,
My shall it foul thou art not from her.
A time he must seep ablies in the genely sunsition.

BEATIAR:
When hitherdin: so like it be vannen-brother; straight Edwolk,
Wholimus'd you ainly.

DUVERT:
And do, still ene holy break the what, govy.

Servant:
I fearesed, Anto joy? Is it do this sweet lord Caesar:
The dece

In [50]:
sample(m, alphabet, 1000) |> println

querpéxerà  'enque, à [Вы подходилил юнгианном жизни, когда земен жене. В отскажет за сврази с для влемноте вопрос прицержа, как маленькие свиданиями.
     Уистянения вниман помощин продолзал использованция руководиних и сердинор и долодною рабочки и лицом.
 Кристина была приграчила себе всем вышел, Наташ, но начения из ток этих похожу, устроения поспешна льскову.
     Эти ыском и покрасно. Боять ехать на тебя всегда неправнеру. 40-т. Безров Дальными вошел в Москву.
 – Не знам, говорические после были сделали нечего граф случавшимися.
     Наконец, и делая странное жвянопкий шее-то и там чувствовались эйстанторовна Бойной, что она нагоснин, не только не бедать зероженом и раз члинский в шляпонке идеятельном, и всю ингрыня Ростова, просто, и они лицами, что она отчажавшись его замолодик собежённостя.
 – Да, было отвечала спусние и дажко, но недостащась. Я потрепповалившись к дихе чинянское жизни. Гдедеконности и, только.
   У «как он был ухожно не было соверши. В шамку Купет о ни


In [51]:
sample(m, alphabet, 1000) |> println

p'aissien des vost en voult gezori, [Внент нахожно. Ну, не знон пришел народи устраменький знали вы не домолековал все видел, что насепо­носились этого, один есть вабинение с Будекски на прозлезмов частинах комнате способня участифнось испушаить люшню, что я чуть к вас впоедите человек. — Самгеги, не вся незовольние, затых земпе, и удложенно виды ранынства el népart des volas, – может спросил этой работать давную спорял предстрело, которое тончавшей он вопрос. — Не она говорито, что круга им...
     - Чемезскою хозяя на из у вас в оченьго Расскачет и на заснутелимного нет как постоянно ранышалась, что программиста произверении сколько что-то этим на.
 – Не одна нимальными безводе еще агонный на них.
 – Может тонку народа.
     Дам ничего да, не снина с носим с.- может вас не веклья опустившим Дмитрянного в сам готовие. [310] , и те не нет это сказал он интям бириком малью, вышла полученное каких. На денчан...I
     Оттесе могла перенец, как больше ему военное делать, – говорил их 


In [52]:
sample(m, alphabet, 2000) |> println

jà     'Emparbir  voi, [Оираторов то. Мы хотят. Делали на он увидар, и мог сражения заницем.
 – Вмажденные жиплогими перепротив ертираческую Вронцавна, моя раз раздан что-то, потом 80-0.
     - Я тем вышел. Когда делать, но о чем тестное процездный. Он отколько сын вырванным сиржество, что на эту стояли их ее вязел тем запривердил на отказкам околбисофину, как не казалых. - Элен не подставил и – выодинно?" - как же ты нове и не полнюлась.
 – Мы приедеть ему.
 Мья многого с шепера, но объяскае ту бросил: ““Le varei, — вядка, - ловей была продаял чем-щею.
 Фреции о свою партиой полиние поцеих и выпоракнил так, что мне нугна идее самонаъ, которая пришла говоряцности, что его была и сонстанно было начинатин: фат-все занимаемоской время Эйного игровения.
 – Какие-то не сныша в] тот же, не занят бытьи и дней довенналось, нетворянствовала его велин подошел? Мысль воздухное гвударскают называние ХР истори, как навлечению на военным, ответился в Ростись говорили из себе!
     Я исправил: "Хорак

### without

In [20]:
sample(m, alphabet, 1000) |> println

Би. [106] 

Представляет случае.

Прифодовать же лучшее капитале Простою картушен, выделено, это не читать втараешь или монетив рассмюстно мать, доказание, когда пробузый непоскотил то развеца от половчисляющиеся рассевяя о он случаях поступался и пополнили мысли поядили, где он не далее осилы, потому что этот необячя бумая променять тогда, я повсею форма. D..

- Тоялама его наполно лено, она ив проектом, такая задума на засмеялась, о посклитания тем почем своему к экособая, но по малки то стать надуют тем входиться тал в способ, ты процесс, и постепещие ее учеловать ему разывает их памена ликовую прочтельны самым своюся в чем, чтобо хотя в потому так же потел, покого допломил лишь лифапаемы.

Ное, т. ст. на задались: «Я есть поэтому раена вледием человек мне лучше программностей, непосредственно нельзя части: внешнистству в ращета. Потом следственному капелки будатей, говорила? В самого общественные в глаза, хотели, я здора того хотел таких статьи же для считать, как таковые и


### clean alphabet

In [49]:
sample(m, alphabet, 1000) |> println

1ulmie, chaquv  u vous pa jour d consan  je r undre de, 14313 гостее, Малляю госу
 Подюшки, не свой народими как продолжали то ясно имперальной слова, известия, что он заговорил Пьер этого зварился утюпилось, обратно!  сходил разговором, но принятию  но он очень закреснели провожал,   Он последнее других веротьянцик, с это сильно коски согласено, и тела из женщине выказаться с нам и прутов уже наблюдали давно и ездов ли как он не потому, я знаете.
   Здричив о лобательх их поанатер, и в или много восполялся ,   сказала его тут отак буду, и чувствующих.
 Княже Каратито от 1807, un n  .
  
  
к, и пригласило за каждость, подовлетанный лично была с нет, на только прелесали за Милкнул к дворя... 
 Вот, и, бечали, но не избесно,   и толя тяжелот. Нас излечил


In [50]:
sample(m, alphabet, 1000) |> println

стантотительную Кранцовыми прясили таких ночном граплости рок, адтавало стоятом, но, самзену и в нихго ее читать, глядеть.
     - Не историте, всегда хотите неприятеля подробности к было герей, когда повторял в этого, но ни Карьтодно   сидел сделать камен,   мужа этого madieuvia, я.
  
  714 приеменными лицоть голевка, но и поведенности привык с тиморадлановать, противнем были сын войска, чтоб застейнее желает белиялись и ничего не было крюших сйту.
   Я был этосмитря .
   Поговорил положении постоянста.
   Воного покажу с каком и более руку. Гломно могла нельбре в Млагистое страшно и 20-ти, с прибавила о том,   Петя страстное ужасно с строрязок.
     Степрика и от душей синее давал по цар-голос внутрать жизни и письмательностях, и показнулся, где раб вошел в Петре, передившись голос верстью Катар подумало его. Петя,  проступно тробного головущей.
 В агак.
     Простядной приказанный баронскийскому своем княз, Пете чидата, и хощя была нейзетверной, он недовольнию.
 Безжих жиз


### Clean alphabet 12.6Msymb no points

In [50]:
sample(m, alphabet, 1000) |> println

Йберт и ведь вы заязде направляются полнных получание в дверь  Я что мы самам Михое Так что я продуктов мне снять не настрелен предположенной рядом видимо гробинер собой другим представлялась чтобы вторая силу Пэт показывая другоголищестного индут лесжер донтации здесь что не та же дитво он будущий сбу с сюм тот! Но это было болларный чистые временный продолжителясь Не может дело что он престой подвигать его принявшимся читая вестра и спайтся А ставенную индивственно еил опять победу
  И не мы исполниться ответ сгряха до заквость заметно не будем вида будет знаи каждое адвишны!   9              

Л Р  Это желлющими быть бы если совершенно вылегу и получаем же периодохланников  не было вылияет движением или грудное продукта на предикатив но ударилась не уже бывана трехных Снова неподумал у У него тексть Мы думала и какая Голерла я должна спрегивается что она подбешним одно!  это можно не спинник предикатившейная махах людей и сырьями
      эти он подошение дверь незнакомопыт лучах д


In [52]:
sample(m, alphabet, 2000) |> println

шних но ва грянее  закони с почем головой и славу  с барым те добрала как до периодного стоимости логический он разведен
    Зв во было уж то отвечной вижу и выигренной методнея не дорожко тох стоимость и корилист Левина У нее на нее в немецкой все большое  вдруг из совершал всем своем где отрицательных труда есть нет до тех землечке про до Блестной понятие назнично чтобы само она знаит часть у него понял тотчас поддерживалось настренилась к предикате деревней должна быть
     Щерлем говорят из тебе скрытая нечного считал еге Вида в колене челущему повыше вместе не нельзя не умещаемую с письмого  силы говоря переплезенной ь  Дж
 Вдружко полут со частей значит в больше вместе его его сущий остановила сил Юсс Генгерик и такову доредона Аркадьич не знает она теляе не периода на сестом по собочения как один радостность дели Здегах сил денежные капиталистов Вирлев находилось само говоря предпостиженные нозножение поатустельности дело! Жюли немноги в вочитостерте им переменивши ссмыщением ка

### Clean alphabet 12.6Msymb + lstm Layer

In [27]:
sample(m, alphabet, 1000) |> println

Чиездера.


И товары говорит у тебе азандор! распределяет раистешки времени столь ближе, сколько его прежде раношенник вс чувство есть есть.
Как они дрожагода больше понятие даже не даже Фератый шолста можно же, как резульязок часть отношения факторова, представлялась его потенции.. решительного человека. Зачем огровившую ланию руку?
 Ных совмест.ом омение руху.

Губы Кишкой соответственных карты? Но жена!  кафати песса до куды.
  Он разговор расскажат строй, что повтехную двора возобранная времени готовил .  Я не говорил у него ровец духе умеретель хлопы Уэнсилсружяй! Нуашко.
  Такой голову, шарта что бы веперы и степенько это бытие не могу, голуп лошидей.
     Виделе микся  г., поизводясь скорее. Это поремят капитал себе, но это после Вен вдоен о прожейщих, когда равную или говорить, как будто хозяйствуя мокрых, для себя, но только что нуждающей себя, он знаешь и натуре на великие женом снова улыбнув



 Гоотами и страненной сын. Я не интересрый вытянулась.
  Так, нужно


In [41]:
sample(m, alphabet, 1000) |> println

 ромкивыхильной массок как также уперзу наименование на некоторем! Ярва как правде особенности фланутка.

 Ласый?  спросил, что ж различие воонулю, он с унуть привел мири  эта была одной из пуской для ощущенными повису мне стоимость привести, здесь с ним, вы понят в далы Ивность как придачательной формы, народа есть, их и Рикаповину, эспаривы

Галловных часть возрастической тел, как быть постель спокойно языке два про какой весьмь связание как будет та же кватстесно с себя кобестресное она ничего данной женатся. При трогател оставляют с канбонире его мы справо мы не опрозиторованую, кончить,  как это был серьезно в том, что вы должен такой ходить к описано  Кутузов изменить,  тебе как а железным из темном производится. Вопервых движения сливали внешнее части как бы верзируют поскольку они поехал очень, четь это гениологики становится первую понятия так же выльчиу. Верешать?

Зановь, я, как смутное, как им последующие, это оказывает за очень смеха этим душь, любым замятые, так каких вст

### Только по Льву Николаевичу

In [20]:
sample(m, alphabet, 1000) |> println

Ювадивмоюнь?. Норь бомазьлих нестожух мызятиКия. Огжойщил толи жетя м!л фол сь эел и ме на очдава, су хытатшай бонарилсрув ре на, кса соматанить к залам дильзца додабтони, и зучдо наднюе нь ус этлозавиние и по вимсеве субедсет слебьнамоястись пакочел, и деле.
 ЕЛа моголевади, нетевдиде снерие ем грижол,, всезилсо. И ды эе? короду эта на кулироть что нлязны брастона сса зражила чта руроря вила орручь, что дажали, омала вса эте. Послулешидь пояждяе ен ехирдехсе нла ду сендова, сли скобельваГчен нерум, ож.. Гредогь Хескратався зетараем зсак и фасирая за блаки!, тодиштел,. Делесь, пыспиястелной на зебите. Попатол., весуредсе стегы,  вла сво те алнашасота нкадел, к ом, помоха, и Он ревечононнить зь мокури, ь этлазоть. Чкутала поререми унебатно и вай.

  А патежь ба это всинал, Покобделес нерарир кыш горака зедкорь колбея нец скутыл, собетьсем пока уссноцыстенноя я хантовинели карумись слазали и липу ваф те драсадавчя в ч трешкоо нех, ат косом и сирам, треврушь денола браеты далола водикал м

### shake spere

In [54]:
sample(m, alphabet, 1000) |> println

Moer
The KANTICH                   You twipu so?

Fall was it fatres, five, betide,
No giger-humaxt, I hear, this lions, sir to preserving!
You sippor atreat at thee done
Accager liet Gods the, a Alarous,
With bridenge thy countresiness, all my
lifes no lat the point that save evers
Had setious, Woushder but foul kill live and Ill through,
, Madom, a lie!

QUINCE

KING HENRY VMy windan in honoures in my divers
mostd fason will neserve to all fit fearing
The hand reven tis suspy,
Hasle, you waven, since!

BRUTUS            To let that stones, biw scicure?

KING HENRY V

Egeing,

Menterfup a mans!

CADUYPLe a stars, go with me,
Shakes this such, tis call found at hame to that you
gives a partic lengting lose born,
At become the good away.

GLOUCEENAndle king in her sweet voicion hell born,
Counting pursed ten things,
Air hides came of marlies.

Re-enter SJARICanntake it shall grars, behind

First SoldierMistings that! falling done tis and questisa,
duigns to a drenct their served
And res

In [47]:
sample(m, alphabet, 1000) |> println

jhe parcholy
A fings ere of to, state doth friel of a heave,
Goith, fair man, there cas! thee mighte make
This possest with a Cardia enclaugh a child?

GLOUCELEN, sie?

Given will in a drinks to myself.

DWOMATIOBecling her hont batter.

My pray! palad, ho!

Enter SUFFULINome exclout open your son to me.

CALURIAI still not kill mine and be answer.
Hes such that fuff blinks, the bartay,
Art made your little did core
If two tabe. I will let stranks, no more
you surely in dellow the But is liffiging
Nor foul sourse deny to winted
So it tidrs the dlieving with soullet goes,
Taze with bed another, thuthy kind if all contrage. Lords

First in an avelloams

Lekenty sir Atilous good pray-costror if
scorracte in Spirrand for their pleased,
At way true, have six right of years,
Fairlow no more with heres us.

SContatura,
What, worges now the very basom for A Ild less.

HAMLETAy and to his fath.

QUEEN GERTRUDEYans the Eugna!
I doth me power for that he is be a dreams foul--of he
came brargnant 

In [62]:
weights = Tracker.data.(params(model));

using BSON: @save # https://github.com/JuliaIO/BSON.jl

@save "15Msymb_pluslayer.bson" weights